In [2]:
from roboflow import Roboflow
import cv2
import numpy as np
import base64

# Initialize Roboflow
rf = Roboflow(api_key="tYOmMkYZj1W6AScfD6I2")
project = rf.workspace().project("metal_plate")
model = project.version("1").model

# Infer on a local image
local_image_path = "/home/bubbles/Downloads/OK2.jpg"
prediction_response = model.predict(local_image_path, confidence=40).json()

# Get the predictions from the response
predictions = prediction_response.get('predictions', [])

# Check if predictions are present
if predictions:
    # Load the original image
    original_image = cv2.imread(local_image_path)

    # Initialize a counter for the number of holes
    num_holes = 0

    # Iterate through predictions to find holes
    for prediction in predictions:
        points = prediction.get('points', [])

        # Check if there are enough points to form a hole (at least 3 points)
        if len(points) >= 3:
            # Increment the counter for each hole found
            num_holes += 1

            # Optionally, draw a bounding box around the hole for visualization
            rect = cv2.boundingRect(np.array([(int(p['x']), int(p['y'])) for p in points]))
            cv2.rectangle(original_image, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)

    # Save the image with bounding boxes
    cv2.imwrite("prediction_with_holes221.jpg", original_image)

    # Print the number of holes found
    print(f"Number of holes found: {num_holes}")
    print(f"Holes Missing : {16 - num_holes}")
else:
    print("No predictions found in the response.")


loading Roboflow workspace...
loading Roboflow project...
Number of holes found: 16
Holes Missing : 0


In [1]:
from roboflow import Roboflow
import cv2
import numpy as np
import base64
import tempfile

# Initialize Roboflow
rf = Roboflow(api_key="tYOmMkYZj1W6AScfD6I2")
project = rf.workspace().project("metal_plate")
model = project.version("1").model

# Access the webcam (assuming it's the default camera, change the argument if needed)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Infinite loop to capture frames from the webcam
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale if needed
    # gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Create a temporary image file
    temp_img_path = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False).name
    cv2.imwrite(temp_img_path, frame)

    # Perform inference on the current frame
    prediction_response = model.predict(temp_img_path, confidence=40).json()

    # Get the predictions from the response
    predictions = prediction_response.get('predictions', [])

    # Initialize a counter for the number of holes
    num_holes = 0

    # Iterate through predictions to find holes
    for prediction in predictions:
        points = prediction.get('points', [])

        # Check if there are enough points to form a hole (at least 3 points)
        if len(points) >= 3:
            # Increment the counter for each hole found
            num_holes += 1

            # Optionally, draw a bounding box around the hole for visualization
            rect = cv2.boundingRect(np.array([(int(p['x']), int(p['y'])) for p in points]))
            cv2.rectangle(frame, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)

    # Display the frame with bounding boxes
    cv2.imshow("Webcam with Predictions", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


loading Roboflow workspace...
loading Roboflow project...
